# 쿼리 라우팅 2.
- 리트리버/쿼리 레벨 라우팅 기능
- 대표적 사용새: 요약용 서치엔진 + 일반 본문 시맨틱 서치엔진 나누어서 입력 쿼리 형식이 뭔지에 따라 가변적으로 작동하도록 함.

In [ ]:
!pip install openai llama_index qdrant_client llama-index-vector-stores-qdrant datasets

In [ ]:

import nest_asyncio

nest_asyncio.apply()

In [ ]:
# set up OpenAI
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore

import qdrant_client
from qdrant_client import models
client = qdrant_client.QdrantClient(
    url="",
    api_key="",
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

# llm 모델이랑 임베딩 모델 글로벌 세팅
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small"
)
Settings.llm= OpenAI(temperature=0,model='gpt-4o-mini')

In [ ]:
from llama_index.core import SimpleDirectoryReader

# Paul Graham 에세이 다큐먼트 로딩 (폴더 만들고 에세이 파일 업로드 하세요)
documents = SimpleDirectoryReader("/content/pg").load_data()

In [ ]:
# 청크사이즈 조정
Settings.chunk_size = 1024
nodes = Settings.node_parser.get_nodes_from_documents(documents)

In [ ]:
from llama_index.core import StorageContext

#벡터스토어 컬렉션 생성
vector_store = QdrantVectorStore(client=client, collection_name="routing_exercise")
storage_context = StorageContext.from_defaults(vector_store=vector_store)


In [ ]:
from llama_index.core import SummaryIndex
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext

#Summary Index: 서머리 담는 용도의 인덱스타입
summary_index = SummaryIndex(nodes, storage_context=storage_context)

#VectorStoreIndex: 일반적인 덴스임베딩 + 메타 담는곳
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)

In [ ]:

# as_query_engine의 response_synthesize 방식으로 tree_summarize 사용.
# Tree_summarize: retrieved chunk들을 tree구조로 계층적 summarize하여 결과 response 생성하는 방식
list_query_engine = summary_index.as_query_engine(
    response_mode=
)
vector_query_engine = vector_index.as_query_engine()

In [ ]:
from llama_index.core.tools import QueryEngineTool

# 쿼리 엔진 툴로 양 인덱스 각각 등록 및 설명란에 LLM Selector가 셀렉팅 기준 참고용으로 작성
list_tool = QueryEngineTool.from_defaults(
    query_engine=,
    description=(
        "Useful for summarization questions related to Paul Graham eassy on"
        " What I Worked On."
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=,
    description=(
        "Useful for retrieving specific context from Paul Graham essay on What"
        " I Worked On."
    ),
)

## Pydantic Selector
쿼리엔진 셀렉터

LLM Selector : LLM으로 하여금 쿼리엔진툴 description 보고 어느걸 선택해야 하는지에 대한 최종 분류결과를 JSON으로 내뱉게 하고, 이후 이 JSON 바탕으로 쿼리엔진툴 셀렉팅을 하는 느낌.

Pydantic Selector: raw JSON 대신 OpenAI의 Function Call API 대신 활용해서 셀렉션 테스크 수행.

SingleSelector, MultiSelector 옵션으로 활용 가능

In [ ]:

from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector, LLMMultiSelector
from llama_index.core.selectors import (
    PydanticMultiSelector,
    PydanticSingleSelector,
)

# 쿼리엔진툴들을 묶음으로 상위 쿼리엔진인 라우터쿼리엔진 사용.
query_engine = RouterQueryEngine(
    selector=,
    query_engine_tools=[

    ],
)

In [ ]:
# 요약에 관련된 질문
response = query_engine.query("")
print(str(response))

In [ ]:
# 싱글셀렉터가 선택한 쿼리엔진 확인
print(str(response.))

In [ ]:
# 이번엔 구체적인 질문을 날려보기
response = query_engine.query("")
print(str(response))

In [ ]:
print(str(response.))

In [ ]:
# LLM 싱글셀렉터
query_engine = RouterQueryEngine(
    selector=
    query_engine_tools=[

    ],
)

In [ ]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

In [ ]:
print(str(response.metadata["selector_result"]))

In [ ]:
response = query_engine.query("What did Paul Graham do after RICS?")
print(str(response))

In [ ]:
print(str(response.metadata["selector_result"]))

- Multi 셀렉터 (인덱스 여러개 참조해서 리트리브 해야될 경우)

In [ ]:
from llama_index.core import SimpleKeywordTableIndex

# 요약, 구체성에 더해, 이번엔 키워드 기반 추출 엔진도 정의해보자
keyword_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)

keyword_index_query_engine =

keyword_tool = QueryEngineTool.from_defaults(
    query_engine=keyword_index_query_engine,
    description=(
        "Useful for retrieving specific context using keywords from Paul"
        " Graham essay on What I Worked On."
    ),
)

In [ ]:
# 멀티셀렉터 정의
query_engine = RouterQueryEngine(
    selector=,
    query_engine_tools=[

    ],
)

In [ ]:
# 설명이 같은 두개의 툴들(벡터툴, 키워드툴) 멀티셀렉션하는지 확인
response = query_engine.query(
    "What were noteable events and people from the authors time at Interleaf"
    " and YC?"
)
print(str(response))

In [ ]:

print(str(response.metadata["selector_result"]))

# Do It Yourself

In [ ]:
# Dataset 로드
from datasets import load_dataset

ds = load_dataset("HAERAE-HUB/KOREAN-WEBTEXT", split='train[:20]')
data = ds.to_pandas()

In [ ]:
# Document 오브젝트로 변환
from llama_index.core import Document, VectorStoreIndex
docs = []

#Iterative하게 Document 만들기
for i, row in data.iterrows():
    docs.append(Document(
        text=row['text'],
        # extra_info={'title': row['title']}
    ))

In [ ]:
# 청크사이즈 조정 - 1024


# 노드로 닥스 분할
nodes =

In [ ]:

# 쿼드란트 벡터스토어 컬렉션 생성
vector_store = QdrantVectorStore(client=client, collection_name="routing_exercise2")

# 스토리지 컨텍스트에 백엔드로 쿼드란트 벡터스토어 연결
storage_context =

In [ ]:
#Summary Index: 서머리 담는 용도의 인덱스타입
summary_index =

#VectorStoreIndex: 일반적인 덴스임베딩 + 메타 담는곳
vector_index =

#키워드 인덱스
keyword_index =

In [ ]:
list_query_engine = summary_index.as_query_engine(

)
vector_query_engine =

keyword_index_query_engine =

In [ ]:
# 쿼리 엔진 툴로 양 인덱스 각각 등록 및 설명란에 LLM Selector가 셀렉팅 기준 참고용으로 작성
list_tool =

vector_tool =

keyword_tool =

In [ ]:
# 라우터쿼리엔진(멀티셀렉터)으로 묶어주기
query_engine =

In [ ]:
# 질문해볼거 찾기
ds.to_pandas()

In [ ]:
# 요약 질문해보기 : "캐나다가 3년 연속 지구상에서 가장 주목할만한 국가로 선포된 이유에 대해서 요약해봐"
response =
print(str(response))

In [ ]:
# 셀렉터 결과 확인
print(str(response.metadata["selector_result"]))

In [ ]:
# 그냥 질문해보기 : "발틱함대가 격파된 전쟁 이름이 뭐지?"
response =
print(str(response))

In [ ]:
#셀렉터 결과 확인
print(str(response.metadata["selector_result"]))

In [ ]:
# 그냥 질문해보기:  "낙동강 댐 개수"
response =
print(str(response))

In [ ]:
# 셀렉터 결과 확인
print(str(response.metadata["selector_result"]))